In [1]:
import pandas as pd
import json
import time
import resource 

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_qdrant import Qdrant

import qdrant_client
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance


/workspaces/dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PATH = '../data/salesItem.csv'
with open(PATH) as f:
    for _ in range(5):
        print(f.readline())


"SalesItemId";"SalesItemCode";"SalesItemName";"ActiveIngredientsName";"Formularium";"OrganizationId"

"19000000062";"MCD00062";"2 MTM BODY LOTION (DR MUTTY)";" KOSMETIK";"NORMAL";"19"

"100000000146092";"KJD00002";"2-4 (ZWAVEL ZALF) 30GRAM - N/A";" SULFUR, SALICYLIC ACID";"NORMAL";"3,31"

"100000000146091";"KJD00001";"2-4 ZALF /GRAM - N/A";" SULFUR, SALICYLIC ACID";"NORMAL";"3,5,31"

"100000000142499";"DT00146R";"3TC 150MG TAB";" LAMIVUDINE";"NORMAL";"3,17,27,32,43"



In [4]:
df = pd.read_csv(PATH, sep=';')
df.head()


,SalesItemId,SalesItemCode,SalesItemName,ActiveIngredientsName,Formularium,OrganizationId
0,19000000062,MCD00062,2 MTM BODY LOTION (DR MUTTY),KOSMETIK,NORMAL,19
1,100000000146092,KJD00002,2-4 (ZWAVEL ZALF) 30GRAM - N/A,"SULFUR, SALICYLIC ACID",NORMAL,"3,31"
2,100000000146091,KJD00001,2-4 ZALF /GRAM - N/A,"SULFUR, SALICYLIC ACID",NORMAL,"3,5,31"
3,100000000142499,DT00146R,3TC 150MG TAB,LAMIVUDINE,NORMAL,"3,17,27,32,43"
4,117701,LVD00001,3TC-HBV 100MG TAB,LAMIVUDINE,NORMAL,"2,3,4,5,6,10,14,15,17,19,32,37,39"


In [5]:
df.Formularium.value_counts()

Formularium
NORMAL      6276
BPJS        2856
INHEALTH    1805
Name: count, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10937 entries, 0 to 10936
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SalesItemId            10937 non-null  int64 
 1   SalesItemCode          10937 non-null  object
 2   SalesItemName          10937 non-null  object
 3   ActiveIngredientsName  7257 non-null   object
 4   Formularium            10937 non-null  object
 5   OrganizationId         10937 non-null  object
dtypes: int64(1), object(5)
memory usage: 512.8+ KB


In [7]:
df['SalesItemId'].duplicated().sum()

np.int64(0)

In [8]:
df.sample(10)


,SalesItemId,SalesItemCode,SalesItemName,ActiveIngredientsName,Formularium,OrganizationId
6962,14000000010702,LVD02322,KABITARINE 100MG/ML INJ,CYTARABINE,NORMAL,"2,3,7,9,11,15,19,34"
9356,1000012481,D0000786,S-26 HMF GOLD 1GRAM SACH,LACTOSERUM,NORMAL,"2,3,5,13,14,17,22,28,33,36,37,41,43,44"
3053,14000000014406,DPD00248,GRIVIN FORTE 500MG TAB,GRISEOFULVIN,NORMAL,"6,14,16"
5409,103401,D11679,js/ ALPRAZOLAM 1MG TAB,ALPRAZOLAM,BPJS,"5,6,7,17,19,21,22,28,31,32,36,37,41,42,43"
8881,10792,D10792,PREGTENOL 5MG TAB -N/A-,ALLYLESTRENOL,NORMAL,"6,7,17,30"
3520,2000000090102,KJD01141,in/ BONEVELL 6MG/6ML INJ,NaN,INHEALTH,"3,5,17,19,22,27,48"
10690,2000000285201,LVD02493,VIVENA-12 INJ,NaN,NORMAL,"2,13,19,20,27,31,34,36,48"
8778,10776,D10776,PLASBUMIN 20% 100ML INF,HUMAN ALBUMIN,BPJS,"2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,19,20,21,..."
7934,116966,LVD00966,NARFOZ 8MG TAB,ONDANSETRON,NORMAL,"2,3,4,5,6,9,12,13,14,15,16,17,19,20,21,22,23,2..."
6395,105601,D11699,"js/ NOTRIXUM 25MG/2,5ML INJ -N/A-",NaN,BPJS,"7,8,19"


In [9]:
df.shape


(10937, 6)

In [10]:
sales_data = pd.read_csv(PATH, sep=';').to_dict(orient='records')
sales_data[0]

{'SalesItemId': 19000000062,
 'SalesItemCode': 'MCD00062',
 'SalesItemName': '2 MTM BODY LOTION (DR MUTTY)',
 'ActiveIngredientsName': ' KOSMETIK',
 'Formularium': 'NORMAL',
 'OrganizationId': '19'}

In [11]:
# Initiate FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings(
    cache_dir="../embedding_cache",
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

url = "http://localhost:6333"
collection_name = "sales-item"
distance = Distance.COSINE
dimension = 768


/workspaces/dev/venv/lib/python3.12/site-packages/langchain_community/embeddings/fastembed.py:109: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  values["model"] = fastembed.TextEmbedding(


In [28]:
# Initiate client.
client = QdrantClient("localhost", port=6333)

# Show collections.
collections = client.get_collections()
print(collections)


collections=[CollectionDescription(name='faq-question'), CollectionDescription(name='sales-item')]


In [13]:
# # Show collection details.
# collection_info = client.get_collection(collection_name="test_salesItem")
# print(collection_info)

In [ ]:
# Delete collection.
# client.delete_collection(collection_name="sales-item")


True

In [31]:
def moveEmbedding(sales_data, batch_size=100):
    """
    Load sales items data from a list of dictionaries and upsert embeddings into Qdrant,
    using batch processing to speed up embedding generation.
    
    Setiap item data penjualan harus memiliki key:
      - 'SalesItemId', 'SalesItemName', 'SalesItemCode', 'ActiveIngredientsName', 'Formularium', 'OrganizationId'
    """
    # Initialize Qdrant client.
    client = qdrant_client.QdrantClient(url=url)
    
    # Create collection if it doesn't exist.
    if not client.collection_exists(collection_name=collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=dimension, distance=distance),
        )
    
    total_batches = (len(sales_data) - 1) // batch_size + 1
    overall_start_time = time.time()
    
    for batch_num in range(total_batches):
        batch_start_time = time.time()
        start = batch_num * batch_size
        end = start + batch_size
        batch_sales_data = sales_data[start:end]
        
        texts = []
        ids = []
        payloads = []
        
        # Build texts, IDs, and payloads for the current batch.
        for item in batch_sales_data:
            text = str(item['SalesItemName']) + "\n" + str(item['ActiveIngredientsName'])
            texts.append(text)
            ids.append(item['SalesItemId'])
            payloads.append({
                "page_content": text,
                "metadata": {
                    "SalesItemName": item['SalesItemName'],
                    "SalesItemCode": item['SalesItemCode'],
                    "ActiveIngredientsName": item['ActiveIngredientsName'],
                    "Formularium": item['Formularium'],
                    "OrganizationId": item['OrganizationId']
                }
            })
        
        # Batch embed the texts using the embed_documents method.
        batch_embeddings = embeddings.embed_documents(texts)
        
        # Prepare points for the current batch.
        points = []
        for j, emb in enumerate(batch_embeddings):
            points.append(
                PointStruct(
                    id=ids[j],
                    vector=emb,
                    payload=payloads[j]
                )
            )
        
        # Upsert the current batch of points into Qdrant.
        client.upsert(
            collection_name=collection_name,
            points=points
        )
        
        batch_end_time = time.time()
        batch_elapsed = batch_end_time - batch_start_time
        print(f"Processed batch {batch_num+1}/{total_batches} in {batch_elapsed:.2f} seconds")
    
    overall_end_time = time.time()
    total_elapsed = overall_end_time - overall_start_time
    peak_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
    print(f"Added: {len(sales_data)} Sales Items in {total_elapsed:.2f} seconds, Peak Memory: {peak_memory:.2f} MB")


In [32]:
moveEmbedding(sales_data[:1000])

Processed batch 1/10 in 4.98 seconds
Processed batch 2/10 in 15.22 seconds
Processed batch 3/10 in 1.97 seconds
Processed batch 4/10 in 1.89 seconds
Processed batch 5/10 in 1.78 seconds
Processed batch 6/10 in 2.04 seconds
Processed batch 7/10 in 4.49 seconds
Processed batch 8/10 in 4.50 seconds
Processed batch 9/10 in 6.73 seconds
Processed batch 10/10 in 9.46 seconds
Added: 1000 Sales Items in 53.06 seconds, Peak Memory: 2908.44 MB


In [33]:
client = QdrantClient(
    url=url,
)
qdrant = Qdrant(
    client,
    embeddings=embeddings,
    collection_name=collection_name,
    distance_strategy=distance,
)


In [34]:
qdrant.similarity_search_with_score(
    query="2 MTM BODY LOTION (DR MUTTY)",
    k=10
)


[(Document(metadata={'SalesItemName': '2 MTM BODY LOTION (DR MUTTY)', 'SalesItemCode': 'MCD00062', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000062, '_collection_name': 'sales-item'}, page_content='2 MTM BODY LOTION (DR MUTTY)\n KOSMETIK'),
  0.9808668),
 (Document(metadata={'SalesItemName': 'ANTI HAIR LOSS MTM SHAMPO (DR MUTTY)', 'SalesItemCode': 'MCD00071', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000071, '_collection_name': 'sales-item'}, page_content='ANTI HAIR LOSS MTM SHAMPO (DR MUTTY)\n KOSMETIK'),
  0.7504275),
 (Document(metadata={'SalesItemName': 'ANTI RASH CP LOTION (DR CAROLINE)', 'SalesItemCode': 'MCD00072', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000072, '_collection_name': 'sales-item'}, page_content='ANTI RASH CP LOTION (DR CAROLINE)\n KOSMETIK'),
  0.71839803),
 (Document(metadata={'SalesItemName':

In [35]:
doc = qdrant.similarity_search_with_score('2 MTM BODY LOTION (DR MUTTY)', score_threshold=0.1, k=10)
for d in doc:
    print(d[0].metadata)

{'SalesItemName': '2 MTM BODY LOTION (DR MUTTY)', 'SalesItemCode': 'MCD00062', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000062, '_collection_name': 'sales-item'}
{'SalesItemName': 'ANTI HAIR LOSS MTM SHAMPO (DR MUTTY)', 'SalesItemCode': 'MCD00071', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000071, '_collection_name': 'sales-item'}
{'SalesItemName': 'ANTI RASH CP LOTION (DR CAROLINE)', 'SalesItemCode': 'MCD00072', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000072, '_collection_name': 'sales-item'}
{'SalesItemName': 'ACNE SPRAY 50ML (DR NETTY)', 'SalesItemCode': 'JRD00007', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '21', '_id': 21000000005, '_collection_name': 'sales-item'}
{'SalesItemName': 'AB-3 CREAM DR INEKE', 'SalesItemCode': 'LCD00003', 'ActiveIngredientsName': ' KOSMETIK', '

In [41]:
test_list = [{'query': x['SalesItemName'], 'expected': x['SalesItemName']} for x in sales_data[:1000]]
print(test_list[0])

{'query': '2 MTM BODY LOTION (DR MUTTY)', 'expected': '2 MTM BODY LOTION (DR MUTTY)'}


In [42]:
test_list = test_list + [
    {
        "query": "vaksin campak dinkes",
        "expected": "VACCINE CAMPAK 4ML INJ (DINKES)"
    },
    {
        "query": "obh batuk berdahak",
        "expected": "OBH BATUK BERDAHAK 100ML SYR"
    },
    {
        "query": "detol cair",
        "expected": "DETTOL CAIR 100ML"
    },
    {
        "query": "antiseptic",
        "expected": "SABUN ANTISEPTIC A (DR.MUTTY)"
    },
    {
        "query": "sirup batuk pilek",
        "expected": "TRIAMINIC BATUK & PILEK 60ML SYR"
    },
    {
        "query": "test",
        "expected": "test"
    }
]


In [45]:
def testing(qdrant, q, expected):
    doc = qdrant.similarity_search_with_score(q, score_threshold=0.5, k=20)
    i = 0
    for d in doc:
        if(d[0].metadata["SalesItemName"] == expected):
            return [ True, d]
        i += 1
    if len(doc) == 0:
        return [False, "No result"]
    return [False, doc[0]]


In [46]:
# With question only.
results = []
for t in test_list:
    [status, data] = testing(qdrant, t["query"], t["expected"])
    if(status == False):
        _data = {
            "query": t["query"],
            "result": data[0].metadata['SalesItemName'] if data != "No result" else "No result",
            "score": data[1] if data != "No result" else "No result"
        }
        results.append(_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    df = pd.DataFrame(data=results)
    display(df)
    

,query,result,score
0,ABBOTIC 500MG TAB,ACETOSAL 500MG TAB,0.852335
1,ABBOTIC XL 500MG TAB,ACETOSAL 500MG TAB,0.830628
2,ACETOSAL 100MG TAB,am/ ACARBOSE 100MG TAB,0.815852
3,ACITRAL 120ML SYR,ACRAN 150MG FC TAB,0.692104
4,ACITRAL TAB,AFINITOR 5MG TAB,0.694709
5,ACTIFED (KUNING) 60ML SYR,am/ ACTIFED 60 ML SYR,0.76494
6,ACTIFED PLUS EXP 60ML SYR,am/ ACTIFED PLUS EXPECTORANT 60 ML SYR,0.76726
7,ACTIFED PLUS EXPECTORANT 120ML SYR,am/ ACTIFED PLUS EXPECTORANT 60 ML SYR,0.719639
8,ACTONEL 150MG TAB,am/ APROVEL 150MG TAB,0.804774
9,ADALAT 10MG TAB,AFINITOR 10MG TAB @30,0.864541
